<a href="https://colab.research.google.com/github/AdityaMali918/Python/blob/main/GEN_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. RunnableConnector self developed  

In [ ]:
from abc import abstractmethod,ABC

In [ ]:
class Runnable(ABC):
  @abstractmethod
  def invoke(input_data):
    pass

In [ ]:
import random

class NakliLLM(Runnable):
  def __init__(self):
    print("LLM created")

  def invoke(self,prompt):
    response_list = [
        'Delhi is the capital of India',
        'IPL is a cricket league',
        'AI stands for Artificial Intelligence'
    ]

    return {"response": random.choice(response_list)}

In [ ]:
class NakliPromptTemplate(Runnable):
  def __init__(self,template,input_variables):
    self.template = template
    self.input_variables = input_variables

  def invoke(self,input_dict):
    return self.template.format(**input_dict)

  def format(self,input_dict):
    return self.template.format(**input_dict)

In [ ]:
class NakliStrOutputParser(Runnable):

  def __init__(self):
    pass

  def invoke(self,input_data):
    return  input_data['response']

In [ ]:
class RunnableConnector(Runnable):
  def __init__(self,runnable_list):
    self.runnable_list = runnable_list

  def invoke(self,input_data):
    for runnable in self.runnable_list:
      input_data = runnable.invoke(input_data)
    return input_data


In [ ]:
template = NakliPromptTemplate(
    template='Write a {length} poem about {topic}',
    input_variables=['length', 'topic']
)

In [ ]:
llm = NakliLLM()

LLM created


In [ ]:
parser = NakliStrOutputParser()

In [ ]:
chain = RunnableConnector([template,llm,parser])

In [ ]:
chain.invoke({'length':'long', 'topic':'india'})

'IPL is a cricket league'

# ***Next Example***

In [ ]:
template1 = NakliPromptTemplate(
    template='Write a joke about {topic}',
    input_variables=['topic']
)

In [ ]:
template2 = NakliPromptTemplate(
    template='Explain the following joke {response}',
    input_variables=['response']
)

In [ ]:
chain1 = RunnableConnector([template1, llm])

In [ ]:
chain2 = RunnableConnector([template2, llm, parser])

In [ ]:
final_chain = RunnableConnector([chain1, chain2])

In [ ]:
final_chain.invoke({'topic':'cricket'})

'IPL is a cricket league'

# **Runnable**

# **1. RunnableSequennce**

In [ ]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from langchain.schema.runnable import RunnableSequence
from langchain_core.output_parsers import StrOutputParser
import os

os.environ['HF_HOME'] = 'D:/huggingface_cache'

llm = HuggingFacePipeline.from_model_id(
    model_id='TinyLlama/TinyLlama-1.1B-Chat-v1.0',
    task='text-generation',
    device=0,
    pipeline_kwargs=dict(
        temperature=0.5,
        max_new_tokens=100,
    )
)

model = ChatHuggingFace(llm=llm)

prompt1 = PromptTemplate(
      template = 'Write a joke about {topic}',
      input_variables=["topic"]
)

parser = StrOutputParser()

chain = RunnableSequence(prompt1,model,parser)

result = chain.invoke({"topic":"traffic"})
print(result)


Device set to use cuda:0


<|user|>
Write a joke about traffic</s>
<|assistant|>
Traffic Cop: "Hey, watch it there! You're going too fast!"

Driver 1: "Uh, sorry about that. I was on my way to work."

Driver 2: "Oh, you're a traffic cop? That's awesome!"

Driver 1: "Yeah, it's my job to keep the road safe."

Driver 2: "That's good to know. I thought


# **2. RunnableParallel**

In [ ]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableSequence, RunnableParallel
import os

os.environ["HF_HOME"] = 'D:/huggingface_cache'

llm = HuggingFacePipeline.from_model_id(
    model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    task = "text-generation",
    device = 0,
    pipeline_kwargs=dict(
        temperature = 0.7,
        max_new_tokens = 150
    )
)

model = ChatHuggingFace(llm = llm)

parser = StrOutputParser()

prompt1 = PromptTemplate(
    template="Generate Linkedin Post on this {topic}",
    input_variables=["topic"]
)

prompt2 = PromptTemplate(
    template="Generate Twitter tweet on this {topic}",
    input_variables=["topic"]
)

parallel_chain = RunnableParallel({
    "linkedin":RunnableSequence(prompt1,model,parser),
    "tweet":RunnableSequence(prompt2,model,parser)
})
result = parallel_chain.invoke({"topic":"AI"})
print(result["linkedin"])
print(result["tweet"])

Device set to use cuda:0


<|user|>
Generate Linkedin Post on this AI</s>
<|assistant|>
Looking for a new career path in the exciting world of AI? Whether you're an experienced professional or just starting out, this is the perfect opportunity to invest in your future and up-skill yourself with the latest advancements in Artificial Intelligence (AI).

At [Company Name], we believe that AI is the future of business and we are proud to offer our employees the opportunity to gain hands-on experience and gain valuable industry knowledge. Our AI training program allows our employees to learn and develop their skills with the latest tools and technologies, while also contributing to our company's growth and success.

Here are just a few of the benefits that you can expect from this program
<|user|>
Generate Twitter tweet on this AI</s>
<|assistant|>
"With the advancements in AI, our world is becoming smarter and more efficient. From self-driving cars to chatbot assistants, AI is transforming how we see and interact wi

# **3. RunnablePassthrough**

### It just returns what ever input you give

In [ ]:
from langchain_huggingface import ChatHuggingFace,HuggingFacePipeline
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableSequence,RunnableParallel,RunnablePassthrough
from langchain_core.prompts import PromptTemplate
import os

os.environ["HF_HOME"] = 'D:/huggingface_cache'

llm = HuggingFacePipeline.from_model_id(
    model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    task="text-generation",
    device = 0,
    pipeline_kwargs=dict(
        max_new_tokens=150,
        temperature = 0.8
    )
)

model = ChatHuggingFace(llm = llm)

prompt1 = PromptTemplate(
    template='Write a one liner joke about {topic}',
    input_variables=['topic']
)

parser = StrOutputParser()

prompt2 = PromptTemplate(
    template='Explain the following joke - {text}',
    input_variables=['text']
)

get_joke = RunnableSequence(prompt1,model,parser)

parallel_joke = RunnableParallel({
    "joke":RunnablePassthrough(),
    "explanation":RunnableSequence(prompt2,model,parser)
})

chain = RunnableSequence(get_joke,parallel_joke)

print(chain.invoke({'topic':'college'}))

Device set to use cuda:0


{'joke': '<|user|>\nWrite a one liner joke about college</s>\n<|assistant|>\n"I went to college and learned a thing or two about being a jerk."', 'explanation': '<|user|>\nExplain the following joke - <|user|>\nWrite a one liner joke about college</s>\n<|assistant|>\n"I went to college and learned a thing or two about being a jerk."</s>\n<|assistant|>\n"She was a sophomore, and she took it upon herself to study the dictionary. The professor was so impressed that he granted her an A."'}


In [ ]:
from langchain_huggingface import ChatHuggingFace,HuggingFacePipeline
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableSequence,RunnableParallel,RunnablePassthrough
from langchain_core.prompts import PromptTemplate
import os

os.environ["HF_HOME"] = 'D:/huggingface_cache'

llm = HuggingFacePipeline.from_model_id(
    model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    task="text-generation",
    device =0,
    pipeline_kwargs=dict(
        max_new_tokens=150,
        temperature = 0.8
    )
)

model = ChatHuggingFace(llm = llm)

print(model.invoke("Explain me this joke.\nI went to the aquarium this weekend, but I didn’t stay long. There’s something fishy about that place."))

Device set to use cuda:0


content="<|user|>\nExplain me this joke.\nI went to the aquarium this weekend, but I didn’t stay long. There’s something fishy about that place.</s>\n<|assistant|>\nSure, here's a retelling of the joke:\n\nYou hear some funny fish noises coming from the aquarium. You decide to visit and spend a few hours exploring the mysterious and fascinating world of fish. As you walk around, you see some amazing creations bobbing around in the tank. The water is crystal clear, and you can see fish with teeth, scales, and other intricate features.\n\nYou spend some time admiring the colorful fish, and you even catch a few yourself by playing around in the water. But then, you notice a strange change. The fish seem to be acting strange, and you're not sure what it is. They" additional_kwargs={} response_metadata={} id='run--fd53fc40-bec2-4440-ae7c-36a4db636f80-0'


#4. **RunnableLambda**

In [ ]:
from langchain.schema.runnable import RunnableSequence,RunnableParallel,RunnablePassthrough,RunnableLambda

def word_counter(text):
  return len(text.split())

runnableLambdaText  = RunnableLambda(word_counter)

runnableLambdaText.invoke("<|user|>\nExplain me this joke.\nI went to the aquarium this weekend, but I didn’t stay long. There’s something fishy about that place.</s>\n<|assistant|>\nSure, here's a retelling of the joke:\n\nYou hear some funny fish noises coming from the aquarium. You decide to visit and spend a few hours exploring the mysterious and fascinating world of fish. As you walk around, you see some amazing creations bobbing around in the tank. The water is crystal clear, and you can see fish with teeth, scales, and other intricate features.\n\nYou spend some time admiring the colorful fish, and you even catch a few yourself by playing around in the water. But then, you notice a strange change. The fish seem to be acting strange, and you're not sure what it is. They")

132

In [ ]:
from langchain_huggingface import ChatHuggingFace,HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from langchain.schema.runnable import RunnableLambda
import os

os.environ['HF_HOME']="D:/huggingface_cache"

llm = HuggingFacePipeline.from_model_id(
    model_id='TinyLlama/TinyLlama-1.1B-Chat-v1.0',
    task='text-generation',
    device=0,
    pipeline_kwargs=dict(
        temperature=0.5,
        max_new_tokens=100,
    )
)

model = ChatHuggingFace(llm=llm)

def word_count(text):
    return len(text.split())

prompt = PromptTemplate(
    template='Write a joke about {topic}',
    input_variables=['topic']
)


joke_gen_chain = RunnableSequence(prompt, model, parser)

## 1st method

# parallel_chain = RunnableParallel({
#     "joke": RunnablePassthrough(),
#     "word_count":RunnableLambda(word_count)
# })

## 2nd method

parallel_chain = RunnableParallel({
    "joke": RunnablePassthrough(),
    "word_count":RunnableLambda(lambda x : len(x.split()))
})

final_chain = RunnableSequence(joke_gen_chain, parallel_chain)

result = final_chain.invoke({'topic':'AI'})

final_result = """{} \n word count - {}""".format(result['joke'], result['word_count'])

print(final_result)

Device set to use cuda:0


<|user|>
Write a joke about AI</s>
<|assistant|>
Q: Can you summarize the joke about AI?

A: A: Sure!

AI: (laughs) "Hey, can you translate this joke into AI language?"

AI2: (smirks) "Sure, I can do that!"

AI: (voice-over) "It's always funny how humans get confused by AI's jokes."

AI: (voice-over) 
 word count - 50


# **5. Runnable Branch**

In [ ]:
from langchain_huggingface import ChatHuggingFace,HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from langchain.schema.runnable import RunnableLambda,RunnableSequence,RunnableParallel,RunnablePassthrough,RunnableBranch
from langchain_core.output_parsers import StrOutputParser
import os

os.environ['HF_HOME']="D:/huggingface_cache"

llm = HuggingFacePipeline.from_model_id(
    model_id='TinyLlama/TinyLlama-1.1B-Chat-v1.0',
    task='text-generation',
    device=0,
    pipeline_kwargs=dict(
        temperature=0.5,
        max_new_tokens=350,
    )
)

model = ChatHuggingFace(llm=llm)

prompt1 = PromptTemplate(
    template='Write a detailed report on {topic}',
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template='Summarize the following text \n {text}',
    input_variables=['text']
)

parser = StrOutputParser()

report_gen_chain = prompt1 | model | parser

branch_chain = RunnableBranch(
    (lambda x : len(x.split())>300,prompt2 | model | parser),
    RunnablePassthrough()
)

def word_counter(text):
  return len(text.split())

runnableLambdaText  = RunnableLambda(word_counter)

final_chain = RunnableSequence(report_gen_chain, branch_chain)

print(final_chain.invoke({'topic':'Russia vs Ukraine'}))

Device set to use cuda:0


<|user|>
Write a detailed report on Russia vs Ukraine</s>
<|assistant|>
Introduction

The ongoing conflict between Ukraine and Russia has been a significant geopolitical and geoeconomic event that has had far-reaching implications for both countries and the global community. This report explores the historical, political, economic, and military dimensions of the conflict, as well as its impact on the international community and the global economy.

Historical Context

The conflict between Ukraine and Russia began in 2014 when pro-Russian separatists in eastern Ukraine declared an independent state called "Donetsk People's Republic" and "Luhansk People's Republic." The following year, Russia annexed Crimea, a peninsula on the Black Sea, from Ukraine. This move was seen as a violation of Ukraine's sovereignty and territorial integrity, and it triggered a series of events that led to the conflict.

The conflict has been characterized by a series of escalating events, including a military 

# **Document Loader**

## Text Loader

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_huggingface import HuggingFacePipeline,ChatHuggingFace
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
import os

os.environ["HF_HOME"] = 'D:/huggingface_cache'

llm = HuggingFacePipeline.from_model_id(
    model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    task = "text-generation",
    device= 0 ,
    pipeline_kwargs=dict(
        max_new_tokens=500,
        temperature = 0.7
    )
)

model = ChatHuggingFace(llm = llm)

prompt = PromptTemplate(
    template="Write a summary for the following poem - \n {poem}",
    input_variables= ['poem']
)

parser = StrOutputParser()

loader = TextLoader("cricket.txt",encoding="utf-8")

docs = loader.load()


chain = prompt | model | parser

result = chain.invoke({'poem':docs[0].page_content})
print(result)


# print(docs)

# print(type(docs))
# print(docs[0].page_content)

# print(docs[0].metadata)

Device set to use cuda:0
Token indices sequence length is longer than the specified maximum sequence length for this model (2482 > 2048). Running this sequence through the model will result in indexing errors


<|user|>
Write a summary for the following poem - 
 Beneath the sun or floodlight's gleam,

Cricket lives like a waking dream.

A field of green, a willowed sound,

Where legends rise and tales are found.

From dusty lanes where barefoot boys,

Chase every run with shrieks of joy,

To packed arenas roaring loud,

The game unites a global crowd.

A coin is tossed, the captains stare,

As tension thickens in the air.

Bat or bowl? A choice so bold,

A story new begins, retold.

The openers walk, calm yet brave,

Each stride a wave upon the wave.

They face the ball with narrowed eyes,

As silence grips the watching skies.

The bowler runs, a rhythmic beat,

Like thunder galloping on feet.

A leather flash, a wooden crack—

The ball takes flight, then tumbles back.

A flick through square, a drive through mid,

A lofted shot the fielder missed.

A single, double, sprint for three,

The crowd erupts in ecstasy.

But not for long—the trap is set,

The spinner loops, the pitch is wet.

A sud

# **PyPDFLoader**

In [ ]:
from langchain_community.document_loaders import PyPDFLoader


loader = PyPDFLoader("/content/dl-curriculum.pdf")

docs = loader.load()

print(docs)

[Document(metadata={'producer': 'Skia/PDF m131 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Deep Learning Curriculum', 'source': '/content/dl-curriculum.pdf', 'total_pages': 23, 'page': 0, 'page_label': '1'}, page_content='CampusXDeepLearningCurriculum\nA.ArtificialNeuralNetworkandhowtoimprovethem\n1.BiologicalInspiration\n● Understandingtheneuronstructure● Synapsesandsignaltransmission● Howbiologicalconceptstranslatetoartificialneurons\n2.HistoryofNeuralNetworks\n● Earlymodels(Perceptron)● BackpropagationandMLPs● The"AIWinter"andresurgenceofneuralnetworks● Emergenceofdeeplearning\n3.PerceptronandMultilayerPerceptrons(MLP)\n● Single-layerperceptronlimitations● XORproblemandtheneedforhiddenlayers● MLParchitecture\n4. LayersandTheirFunctions\n● InputLayer○ Acceptinginputdata● HiddenLayers○ Featureextraction● OutputLayer○ Producingfinalpredictions\n5.ActivationFunctions'), Document(metadata={'producer': 'Skia/PDF m131 Google Docs Renderer', 'creator': 'PyPDF', 

In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 kB 18.1 MB/s eta 0:00:00


# **Directory Loader**

In [ ]:
from langchain_community.document_loaders import  PyPDFLoader,DirectoryLoader

loader = DirectoryLoader('Document-Loaders\Books',glob='*.pdf',loader_cls=PyPDFLoader)

docs = loader.lazy_load()

for document in docs:
    print(document.metadata)

print(len(docs))

# **Web Based Loader**

In [ ]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00


In [ ]:
!pip install langchain_huggingface

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_huggingface import HuggingFacePipeline,ChatHuggingFace
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
import os

os.environ["HF_HOME"] = 'D:/huggingface_cache'

llm = HuggingFacePipeline.from_model_id(
    model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    task = "text-generation",
    device= 0 ,
    pipeline_kwargs=dict(
        max_new_tokens=500,
        temperature = 0.7
    )
)

model = ChatHuggingFace(llm = llm)

prompt = PromptTemplate(
    template="Write a summary for the following poem - \n {poem}",
    input_variables= ['poem']
)

parser = StrOutputParser()

url = 'https://www.amazon.in/ref=PS5BAU25QCPS5fortnitedisc/dp/B08GZ6QNTC/?_encoding=UTF8&pd_rd_w=I3SNY&content-id=amzn1.sym.3b2d0011-a8ec-4315-a031-cca3c26cfcd6&pf_rd_p=3b2d0011-a8ec-4315-a031-cca3c26cfcd6&pf_rd_r=0JM9Z321WH17RVCB54QJ&pd_rd_wg=t2aNx&pd_rd_r=ff94a82f-a4df-4de0-b6aa-7f11cd47de84&ref_=pd_hp_d_atf_unk'
loader = WebBaseLoader(url)


prompt = PromptTemplate(
    template='Answer the following question \n {question} from the following text - \n {text}',
    input_variables=['question','text']
)

docs = loader.load()

parser = StrOutputParser()

chain = prompt | model | parser

print(chain.invoke({'question':'What is the prodcut that we are talking about?', 'text':docs[0].page_content}))


print(docs[0].page_content)































































































Sony DualSense Wireless Controller White (PlayStation 5) : Amazon.in: Video Games










































  Shortcuts menu


Skip to



        Main content
      



        About this item
      



        About this item
      



        About this item
      



        Buying options
      



        Compare with similar items
      



        Videos
      



        Reviews
      




      Keyboard shortcuts
  




Search

alt
+
/







Cart

shift
+
alt
+
C







Home

shift
+
alt
+
H







Orders

shift
+
alt
+
O







Add to cart

shift
+
alt
+
K







Open/close shortcuts menu

shift
+
alt
+
Z










To move between items, use your keyboard's up or down arrows.















.in









                   Delivering to Mumbai 400001
                

                   Update location
                




















Video Games


Select the depa

# **CSV Loader**

In [ ]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader(file_path='Social_Network_Ads.csv')

docs = loader.load()

print(len(docs))
print(docs[1])

400
page_content='User ID: 15810944
Gender: Male
Age: 35
EstimatedSalary: 20000
Purchased: 0' metadata={'source': 'Social_Network_Ads.csv', 'row': 1}
